In [3]:
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertForTokenClassification

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [4]:
data = pd.read_csv('set_index.csv')

In [5]:
data.head(5)

,input,task_indices,date_indices,time_indices
0,schedule a meeting with john tomorrow at 2 pm,"['O', 'O', 'B-task_detail', 'I-task_detail', '...","['O', 'O', 'O', 'O', 'O', 'B-date', 'O', 'O', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-time', ..."
1,remind me to buy groceries on friday at 6 pm,"['O', 'O', 'O', 'B-task_detail', 'I-task_detai...","['O', 'O', 'O', 'O', 'O', 'O', 'B-date', 'O', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ti..."
2,schedule a call with sarah for next monday at ...,"['O', 'O', 'B-task_detail', 'I-task_detail', '...","['O', 'O', 'O', 'O', 'O', 'O', 'B-date', 'I-da...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
3,set a reminder to submit the project report by...,"['O', 'O', 'O', 'O', 'B-task_detail', 'I-task_...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
4,remind me to water the plants on sunday morning,"['O', 'O', 'O', 'B-task_detail', 'I-task_detai...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-date', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ti..."


In [6]:
task_indices = data['task_indices'].tolist()
time_indices = data['time_indices'].tolist()
date_indices = data['date_indices'].tolist()
input_statements = data['input'].tolist()

task_indices = [eval(elem) for elem in task_indices]
time_indices = [eval(elem) for elem in time_indices]
date_indices = [eval(elem) for elem in date_indices]

In [8]:
print(task_indices[0])

['O', 'O', 'B-task_detail', 'I-task_detail', 'I-task_detail', 'O', 'O', 'O', 'O']


In [8]:
print(len(input_statements))

100


In [52]:
# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize input statements and encode labels
encoded_inputs = tokenizer(input_statements, add_special_tokens=True, padding='longest', truncation=True,return_tensors="tf")

# Convert labels to integer indices
label_to_index = {"O": 0, "B-task_detail": 1, "I-task_detail": 2, "B-date": 3, "I-date": 4, "B-time": 5, "I-time": 6}
# Separate encoding for task, date, and time entities and mapping them according to label_to_index
encoded_labels_task = [[label_to_index[label] for label in task] for task in task_indices]
encoded_labels_date = [[label_to_index[label] for label in date] for date in date_indices]
encoded_labels_time = [[label_to_index[label] for label in time] for time in time_indices]


# Convert encoded labels to TensorFlow tensors
# encoded_labels_task = tf.convert_to_tensor(encoded_labels_task)
# encoded_labels_date = tf.convert_to_tensor(encoded_labels_date)
# encoded_labels_time = tf.convert_to_tensor(encoded_labels_time)


# print("Encoded Inputs:", len(encoded_inputs['input_ids']))

# print("Encoded Labels:", encoded_labels_date[2])


In [53]:
print(len(encoded_inputs['input_ids']),len(encoded_labels_task) ,len(encoded_labels_date),len(encoded_labels_time))

100 100 100 100


In [54]:
max_length = encoded_inputs['input_ids'][0].shape[0]
print(max_length)

23


In [55]:
# Pad labels with "O" to make them rectangular
def pad_labels(labels, max_length):
    return labels + ['O'] * (max_length - len(labels))

# Pad labels for each entity type
padded_task_indices = [pad_labels(task, max_length) for task in task_indices]
padded_date_indices = [pad_labels(date, max_length) for date in date_indices]
padded_time_indices = [pad_labels(time, max_length) for time in time_indices]

# Encode padded labels separately for each entity type
encoded_labels_task = [[label_to_index[label] for label in task] for task in padded_task_indices]
encoded_labels_date = [[label_to_index[label] for label in date] for date in padded_date_indices]
encoded_labels_time = [[label_to_index[label] for label in time] for time in padded_time_indices]

# Convert encoded labels to TensorFlow tensors
encoded_labels_task = tf.constant(encoded_labels_task)
encoded_labels_date = tf.constant(encoded_labels_date)
encoded_labels_time = tf.constant(encoded_labels_time)

In [56]:
import tensorflow as tf
from transformers import TFBertModel, BertTokenizer

# Load pre-trained BERT model and tokenizer
bert_model_name = "bert-base-uncased"
bert_tokenizer = BertTokenizer.from_pretrained(bert_model_name)
bert_model = TFBertModel.from_pretrained(bert_model_name)

# Define input layers
input_ids = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name="input_ids")

# Get BERT outputs
bert_outputs = bert_model(input_ids)[0]  # Output from BERT's last layer

# Define classification heads for each entity type
num_labels_task = 3  # "O", "B-task", "I-task"
num_labels_date = 3  # "O", "B-date", "I-date"
num_labels_time = 3  # "O", "B-time", "I-time"
dropout_rate = 0.1

# Task classification head
task_logits = tf.keras.layers.Dense(num_labels_task, activation=None, name="task_logits")(bert_outputs)
task_output = tf.keras.layers.Activation("softmax", name="task_output")(task_logits)

# # Date classification head
date_logits = tf.keras.layers.Dense(num_labels_date, activation=None, name="date_logits")(bert_outputs)
date_output = tf.keras.layers.Activation("softmax", name="date_output")(date_logits)

# # Time classification head
time_logits = tf.keras.layers.Dense(num_labels_time, activation=None, name="time_logits")(bert_outputs)
time_output = tf.keras.layers.Activation("softmax", name="time_output")(time_logits)

# Define the model
model = tf.keras.Model(inputs=input_ids, outputs=[task_output, date_output,time_output])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [57]:
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 1), loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
# optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
# model.compile(optimizer=optimizer, loss=tf.keras.backend.ctc_batch_cost, metrics=["accuracy"])



In [58]:
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, None)]               0         []                            
                                                                                                  
 tf_bert_model_5 (TFBertMod  TFBaseModelOutputWithPooli   1094822   ['input_ids[0][0]']           
 el)                         ngAndCrossAttentions(last_   40                                      
                             hidden_state=(None, None,                                            
                             768),                                                                
                              pooler_output=(None, 768)                                           
                             , past_key_values=None, hi                                     

In [59]:
history = model.fit(
    encoded_inputs['input_ids'],
    {"task_output": encoded_labels_task, "date_output": encoded_labels_date, "time_output": encoded_labels_time},
    epochs=50,
)

Epoch 1/50


3/4 [=====================>........] - ETA: 1s - loss: 13.4909 - task_output_loss: 5.1149 - date_output_loss: 4.5276 - time_output_loss: 3.8483 - task_output_accuracy: 0.3736 - date_output_accuracy: 0.3361 - time_output_accuracy: 0.4216

KeyboardInterrupt: 